In [ ]:
!pip install openai
import os
import pdb
import time
import openai
import requests
import pandas as pd 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, f1_score

In [4]:
# change this key frequently
OPENAI_API_KEY = ""

In [5]:
openai.api_key = OPENAI_API_KEY

In [6]:
def label_message(meta_statement, definition, article, prompt, role_user="user"):

    completion = openai.chat.completions.create(model="gpt-4",
                                              temperature = 0, # make sure that all answers are the same, 
                                              max_tokens = 50, # 8,000 is the max token length for GPT3.5
                                              messages=[{"role": "system", "content": "You are a news classifier."},
                                                        {"role": role_user, "content": meta_statement},
                                                        {"role": role_user, "content": f'Read this definition {definition}'},
                                                        {"role": role_user, "content": f'Read this article {article}'},
                                                        {"role": role_user, "content": prompt}]) 
    return completion.choices[0].message

In [7]:
news_articles = pd.read_json("old_data_clean.json")

In [9]:
irrelevant_path =  'relevant_irrelevant_combined_gpt.txt'
with open(irrelevant_path, 'r') as file:
    irrelevant_def = file.read()

In [10]:
research_goal = """We categorize articles that are related to issues of income inequality, changes in income or wealth, general economic conditions."""

In [11]:
## experiment 2: remove "\n" character to save on token count if necessary 
irrelevant_def

"- Articles that are not focused on the U.S.\n- Articles that discuss racial or gender inequality or gay rights in the U.S.\n- Articles where individuals' names are part of a subject term (e.g., Marc Rich).\n- References to popular culture that include part of a subject term (e.g., a movie named “Big Business”).\n- Personal affairs about a single individual that do not comment on broader conditions.\n- Articles about noneconomic elites (e.g., in art or religion or politics, in which high economic status is neither implicit nor explicit).\n- Social class as a predictor of non-economic phenomenon (e.g., health, education, drug use, social mobility, conspicuous and other kinds of consumption).\n- Articles that mainly discuss policy or political debates and particular politicians, but do not contain qualifying text on relevant to inquality, relevant to changes, or relevant to economy.\n- Most articles on affirmative action are considered irrelevant.\n- Articles that discuss stories on famo

In [13]:
cols = news_articles.columns.to_list()

In [14]:
keep_cols = cols[:37] 
keep_cols.append('token_count')

In [15]:
# only binary 1: 
binary1 = news_articles[keep_cols]

In [ ]:
messages = []
max_char = 8000*3 
max_token = 7500 # cushion for definition and meta-statement
for i in range(len(binary1)):
    if i % 100 == 0: 
        print("processing message", i)
    title = binary1.loc[i, 'title']
    content = binary1.loc[i, 'text']
    if binary1.loc[i,'token_count'] <= max_token: 
        content = f'title: {title} \n article: {content}'
    else: 
        content = f'title: {title} \n article: {content[:max_char]}'
    article_prompt = "Is the article relevant? Answer relevant or irrelevant, and why in 1 sentence."
    while True: 
        try: 
            output_message = label_message(meta_statement=research_goal, 
                                           article=content, 
                                           definition=irrelevant_def, 
                                           prompt=article_prompt)
            break 
        except Exception as e:
             time.sleep(30)       # add time lag of 30 seconds when an error occurs 
    messages.append(output_message) 
#     pdb.set_trace()

In [20]:
binary1['output_message'] = [message.content for message in messages]

In [22]:
binary1['output_message'] = binary1['output_message'].str.lower()

In [35]:
def parse_answers(binary1, messages): 
    '''
    parse the answer to binary 0 - irrelevant, 1 - relevant 
    '''
    predicted = []
    for message in messages: 
        if message.startswith("irrelevant"): 
            predicted.append('irrelevant')
        elif message.startswith("the article is irrelevant"): 
            #pdb.set_trace()
            predicted.append('irrelevant')
        elif message.startswith("relevant"): 
            predicted.append('relevant')
        elif message.startswith("the article is relevant"): 
            predicted.append('relevant')
        else: 
            predicted.append('flag')
    binary1['predicted_label'] = predicted
    return 

In [36]:
parse_answers(binary1, binary1['output_message'])

In [38]:
# looks like there is no other type of output for further parsing because flag == 0
binary1.predicted_label.value_counts()

predicted_label
relevant      898
irrelevant    355
Name: count, dtype: int64

In [55]:
# save the output for future comparison if needed 
binary1.to_csv('../data/inequality_dataset_GPT_GPTDefinition.csv', index=False)